In [ ]:
import warnings
warnings.filterwarnings('ignore')
from astropy.io import fits

import sys
sys.path.append('../')
from modules import dendro_cutouts, dendro_dendro, dendro_misc, dendro_props, dendro_plot, dendro_mask

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import astropy.units as au
from astropy import stats
from astrodendro import Dendrogram, pp_catalog
from astropy.wcs import WCS

import os
from astropy.wcs import WCS

In [10]:
# Get Galaxy properties
galaxy = 'ngc0628'
# min_value_sig = [2, 3, 4, 5]
min_value_sig = [3]

sampletable_file = "/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits"
sampletable = dendro_dendro.get_GalaxyProps(galaxy, sampletable_file=sampletable_file)
muscat_table = dendro_dendro.get_MuseProps(galaxy)

hdus = dendro_misc.load_pickle('./analysis/catalogue/hdus_sample_sci.pickel')

[INFO] [get_GalaxyProps] Getting sample table properties for ngc0628...
[INFO] [get_MuseProps] Getting MUSE catalouge properties for ngc0628...
[INFO] [load_pickle] Load ./analysis/catalogue/hdus_sample_sci.pickel


In [11]:
# Load the regions...
regions_file = './analysis/catalogue/sample_muse.reg'
# regions = dendro_cutouts.get_regions(regions_file)

# Load HDUS with smoothed, masked, and non-masked data...
hdus_file = './analysis/catalogue/hdus_sample_withoutdendro.pickel'
muscat_ids_file = './analysis/catalogue/muscat_ids.pickel'

if os.path.exists(hdus_file):
    hdus = dendro_misc.load_pickle(hdus_file)
    muscat_ids = dendro_misc.load_pickle(muscat_ids_file)
else: 
    hdus, muscat_ids = dendro_dendro.get_maskedhdus(hdus, regions)
    dendro_misc.save_pickle(hdus, hdus_file)
    dendro_misc.save_pickle(muscat_ids, muscat_ids_file)

[INFO] [load_pickle] Load ./analysis/catalogue/hdus_sample_withoutdendro.pickel
[INFO] [load_pickle] Load ./analysis/catalogue/muscat_ids.pickel


In [ ]:
# from importlib import reload
# import astrodendro
# reload(astrodendro)

# # Replace variable names with appropriate names

# regionID = 20
# hdu = hdus['hst07_hdus_smooth_masked'][regionID]
# hdu_outmask = hdus['hst07_hdus_smooth_maskedall'][regionID]
# hdu_nomask = hdus['hst07_hdus_smooth'][regionID]

# min_delta_sig = 3 
# min_value_sig = 3
# min_npix = 9

# data_nomask = hdu_nomask.data
# data_outmask = hdu_outmask.data

# wcs = WCS(hdu.header)
# header = hdu.header
# data = hdu.data

# # Calculate statistics for dendrogram
# std = stats.mad_std(data_outmask, ignore_nan=True)  # Get noise
# std = stats.mad_std(data_outmask[data_outmask<20*std], ignore_nan=True)  # Get noise below threshold

# pixsize = np.array([np.abs(header['CDELT1']), np.abs(header['CDELT2'])]).mean() * au.degree
# if pixsize.value==1: 
#     if 'CD1_1' in np.array(header.cards)[:,0]: 
#         pixsize = np.array([np.abs(header['CD1_1']), np.abs(header['CD2_2'])]).mean() * au.degree
#     elif 'PC1_1' in np.array(header.cards)[:,0]:
#         pixsize = np.array([np.abs(header['PC1_1']), np.abs(header['PC2_2'])]).mean() * au.degree

# bmaj = bmin = 0.05 * au.arcsec  # Dummy values 

# min_value = std * min_value_sig  # Minimum value within structure
# min_delta = std * min_delta_sig  # Minimum values between levels within structure (sort of)

# # Running the dendrogram
# dendro = Dendrogram.compute(data,
#                             min_delta=min_delta,
#                             min_value=min_value,
#                             min_npix=min_npix,
#                             wcs=wcs)

# dendro.leaves

# # Provide metadata for table output
# metadata = {}
# metadata['data_unit'] = au.Jy / au.beam  # Dummy unit
# metadata['spatial_scale'] = pixsize.to('arcsec')
# metadata['beam_major'] = bmaj.to('arcsec')
# metadata['beam_minor'] = bmin.to('arcsec')

# props = pp_catalog(dendro, metadata, verbose=False)  # Get table
# props

In [15]:
from importlib import reload
reload(dendro_dendro)

hdus, props_all = dendro_dendro.get_dedro_all_decorator(hdus, min_value_sig, sampletable, muscat_ids)
props_all = dendro_dendro.add_muse_info(props_all, muscat_table)

for key in props_all.keys():
    props_all[key]['flux_corr'] = dendro_props.correct_ha_flux(props_all[key]['flux'], props_all[key])
    props_all[key]['ha_lum_hst'] = dendro_props.calculate_luminosity(props_all[key]['flux_corr']*1e-20, props_all[key]['dist_pc'])
    
dendro_misc.save_pickle(hdus, './analysis/catalogue/hdus_sample_withdendro.pickel')
dendro_misc.save_pickle(props_all, './analysis/catalogue/01_props_all_allsig.pickel')        
dendro_dendro.save_fits(props_all, './analysis/catalogue/')

[INFO] [save_fits] Saved to ./analysis/catalogue//01_props_all_3sig.fits


In [19]:
from importlib import reload
reload(dendro_mask)

dendro_mask.get_ds9regions_circ_decor(props_all['3sig'], outputfile='./analysis/catalogue/%s_trunk_regions.reg' %galaxy)
dendro_mask.get_ds9regions_ellip_decor(props_all['3sig'], outputfile='./analysis/catalogue/%s_sigma_regions.reg' %galaxy)

dir_hst = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/hst_contsub/'
hst07_file = '%s/ngc628c_hst_ha_sic.fits' %dir_hst
hdu = fits.open(hst07_file)[0]

dendro_mask.get_hdumask(hdus, hdu, outputfile='./analysis/catalogue/%s_trunk_mask.fits' %galaxy)

[INFO] [get_ds9regions_circ_decor] Saved regions to ./analysis/catalogue/ngc0628_trunk_regions.reg
[INFO] [get_ds9regions_ellip_decor] Saved regions to ./analysis/catalogue/ngc0628_sigma_regions.reg


Masking regions:   0%|          | 0/2369 [00:00<?, ?it/s]

In [ ]:
# Using the `plot_sigmacomp_decorator` function from the `dendro_dendro_plot` module,
# plot a sigma comparison based on the provided HDUs, dendrogram properties (`props_all`), 
# regions, region ID (`regionID`), and minimum value significance (`min_value_sig`).

# dendro_dendro_plot.plot_sigmacomp_decorator(hdus, props_all, regions, min_value_sig)